In [1]:
# cifar-10 on voting classifier
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
%matplotlib notebook

# data_loading code from the official website
def load_cifar10_batch(cifar10_dataset_fold_path, batch_id):
    file = open(cifar10_dataset_fold_path + '/data_batch_' + str(batch_id), 'rb')
    batch = pickle.load(file, encoding='latin1')
    features = batch['data'].reshape(len(batch['data']), 3, 32, 32).transpose(0, 2, 3, 1)
    labels = batch['labels']
    file.close()
    return features, labels

# load training data
cifar10_dataset_fold_path = '/Users/dayao/Desktop/data_mining/final_project/cifar-10-batches-py'
X_train, y_train = load_cifar10_batch(cifar10_dataset_fold_path, 1)
for i in range(2, 6):
    features, labels = load_cifar10_batch(cifar10_dataset_fold_path, i)
    X_train = np.concatenate((X_train, features))
    y_train = np.concatenate((y_train, labels))
    
# load test data
file = open(cifar10_dataset_fold_path + '/test_batch', 'rb')
batch = pickle.load(file, encoding='latin1')
X_test = batch['data'].reshape(len(batch['data']), 3, 32, 32).transpose(0, 2, 3, 1)
y_test = batch['labels']
file.close()

# normalize the data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
minmax = MinMaxScaler()
X_train_temp = X_train.reshape(X_train.shape[0], 32 * 32 * 3)
X_test_temp = X_test.reshape(X_test.shape[0], 32 * 32 * 3)
X_train = minmax.fit_transform(X_train_temp)
X_test = minmax.fit_transform(X_test_temp)
# X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
# X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

# # one-hot encode the labels
# num_classes = 10
# nums_class = []
# for j in range(1, num_classes + 1):
#     nums_class.append(j)
# lb = LabelBinarizer().fit(nums_class)
# y_train = lb.transform(y_train)
# y_test = lb.transform(y_test)

# shuffle the data to make sure there are no underlying distributions
X_train, y_train = shuffle(X_train, y_train, random_state=624)
X_test, y_test = shuffle(X_test, y_test, random_state=624)

# build voting classier model
clf1 = KNeighborsClassifier(n_neighbors=3)
clf2 = RandomForestClassifier(n_estimators=500, min_samples_leaf=4, min_samples_split=4, 
                              max_depth=None, bootstrap=True, random_state=927, verbose=2)
clf3 = SVC(gamma='auto', verbose=True, decision_function_shape='ovo', random_state=927)

eclf = VotingClassifier(estimators=[('knn', clf1), ('rf', clf2), ('svm', clf3)], voting='hard', n_jobs=4)


In [2]:
print('---------start training---------')
eclf.fit(X_train, y_train)
print('---------stop training---------')

# score the model
predicts = eclf.predict(X_test)
score = accuracy_score(y_test, predicts)
print('Accuracy:', score)

# plot the confusion matrix
confusion_matrix = confusion_matrix(y_test, predicts)
plt.imshow(confusion_matrix)

---------start training---------


C:\my_own_app\Anaconda3\lib\site-packages\sklearn\externals\joblib\disk.py:122: UserWarning: Unable to delete folder C:\Users\dayao\AppData\Local\Temp\joblib_memmapping_folder_13920_6375105210 after 5 tentatives.
  .format(folder_path, RM_SUBDIRS_N_RETRY))


PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'C:\\Users\\dayao\\AppData\\Local\\Temp\\joblib_memmapping_folder_13920_6375105210\\13920-2219816985488-ff9c6e3db8814cb6979c626eddcb7dfc.pkl'